In [ ]:
%run ./00_Mapping_Setup

In [ ]:
/* ===================================================================================
   METRIC: EMP03 - Code of Conduct Timely Training Completion
=================================================================================== */

WITH Training_Data AS (
    -- 1. Evaluate each employee to see if they completed the training on time
    SELECT 
        -- TRY_CAST safely ignores alphanumeric DeptIDs (like 'UYJ4') without crashing
        TRY_CAST(TRIM(DeptID) AS INT) AS Cleaned_DeptID,
        
        CASE 
            WHEN DueDate IS NULL OR TRIM(DueDate) = '' THEN 0
            WHEN CompletionDateUTC IS NULL OR TRIM(CompletionDateUTC) = '' THEN 0
            
            -- FIX: Use to_date with a format string to properly read 'M/D/YYYY'
            WHEN to_date(TRIM(CompletionDateUTC), 'M/d/yyyy') <= to_date(TRIM(DueDate), 'M/d/yyyy') THEN 1
            ELSE 0 
        END AS Completed_On_Time
        
    FROM hive_metastore.ra_adido_2025.td_code_of_conduct_and_ethics_11012024_10312025
    WHERE DeptID IS NOT NULL
),

Mapped_AUs AS (
    -- 2. Join the base AU list to the Training matches
    SELECT 
        m.AU_ID AS BusinessID,
        t.Completed_On_Time,
        CASE WHEN t.Cleaned_DeptID IS NOT NULL THEN 1 ELSE 0 END AS Has_Employee
    FROM vw_cost_center_mapping_bootstrap m
    
    LEFT JOIN Training_Data t
        ON CAST(m.Cost_Center_ID AS INT) = t.Cleaned_DeptID
)

-- 3. Roll everything up to match the Final Output Template
SELECT 
    BusinessID,                          
    'EMP03' AS QuestionID,               
    'Applicable' AS Applicability,       
    '' AS ApplicabilityRationale,        
    
    CASE 
        WHEN SUM(Has_Employee) = 0 THEN '0%' 
        ELSE CONCAT(
            CAST(ROUND((SUM(Completed_On_Time) * 100.0) / SUM(Has_Employee), 2) AS STRING), 
            '%'
        )
    END AS Response
    
FROM Mapped_AUs
GROUP BY BusinessID
ORDER BY BusinessID;